In [29]:
#Import dependencies
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time
import requests
from bs4 import BeautifulSoup
import re
import config
import json
import os
from urllib.parse import urlparse
import csv
from moviepy.editor import *


In [14]:
#specify the path to chromedriver.exe (download and save on your computer)
driver = webdriver.Chrome()

#open the webpage
driver.get("https://www.instagram.com/")

In [12]:
#target username
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

#enter username and password
username.clear()
username.send_keys(config.username)
password.clear()
password.send_keys(config.password)

#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

AttributeError: module 'config' has no attribute 'username'

In [6]:
not_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Not Now")]')))

not_button.click()

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000100aa4200 cxxbridge1$str$ptr + 1907280
1   chromedriver                        0x0000000100a9c6e8 cxxbridge1$str$ptr + 1875768
2   chromedriver                        0x00000001006b0260 cxxbridge1$string$len + 89488
3   chromedriver                        0x00000001006f450c cxxbridge1$string$len + 368700
4   chromedriver                        0x000000010072e7d0 cxxbridge1$string$len + 606976
5   chromedriver                        0x00000001006e912c cxxbridge1$string$len + 322652
6   chromedriver                        0x00000001006e9d7c cxxbridge1$string$len + 325804
7   chromedriver                        0x0000000100a6c490 cxxbridge1$str$ptr + 1678560
8   chromedriver                        0x0000000100a70df8 cxxbridge1$str$ptr + 1697352
9   chromedriver                        0x0000000100a515a4 cxxbridge1$str$ptr + 1568244
10  chromedriver                        0x0000000100a716c8 cxxbridge1$s

In [ ]:
# Wait up to 10 seconds for the search button to be clickable on the web page
search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'svg[aria-label="Search"]')))

# Click the search button once it becomes clickable
search_button.click()

In [ ]:
#target the search input field
searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Search']")))
searchbox.clear()

#search for the @handle or keyword
keyword = "@sample-handle"
searchbox.send_keys(keyword)

In [ ]:
# Check if the keyword starts with "@"
if keyword.startswith("@"):
    # Remove the "@" symbol
    keyword = keyword[1:]
    
# Find the first element with the specified XPath that matches the keyword    
first_result = driver.find_element(By.XPATH, f'//span[text()="{keyword}"]')

# Click on the found element (assuming it represents the desired search result)
first_result.click()

In [23]:
# Get the initial page height
initial_height = driver.execute_script("return document.body.scrollHeight")

# Create a list to store htmls
soups = []

while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait for a moment to allow new content to load (adjust as needed)
    time.sleep(5)
    
    # Parse the HTML
    html = driver.page_source
    
    # Create a BeautifulSoup object from the scraped HTML
    soups.append(BeautifulSoup(html, 'html.parser'))

    # Get the current page height
    current_height = driver.execute_script("return document.body.scrollHeight")

    if current_height == initial_height:
        break  # Exit the loop when you can't scroll further

    initial_height = current_height  # Update the initial height for the next iteration

In [24]:
# List to store the post image URLs
post_urls = []

for soup in soups:
    # Find all anchor elements with href attributes
    anchors = soup.find_all('a', href=True)
    
    # Filter URLs that start with "/p/" or "/reel/"
    post_urls.extend([anchor['href'] for anchor in anchors if anchor['href'].startswith(("/p/", "/reel/"))])

# Convert the list to a set to remove duplicates
unique_post_urls = list(set(post_urls))

print(f"before: {len(post_urls)}, after: {len(unique_post_urls)}")

AttributeError: 'str' object has no attribute 'find_all'

In [ ]:
json_list = []

# Define the query parameters to add
query_parameters = "__a=1&__d=dis"

# go through all urls
for url in unique_post_urls:
    try:
        # Get the current URL of the page
        current_url = driver.current_url

        # Append the query parameters to the current URL
        modified_url = "https://www.instagram.com/" + url + "?" + query_parameters

        # Get URL
        driver.get(modified_url)

        # Wait for a moment to allow new content to load (adjust as needed)
        time.sleep(1)

        # Find the <pre> tag containing the JSON data
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//pre'))
        )
        pre_tag = driver.find_element_by_xpath('//pre')

        # Extract the JSON data from the <pre> tag
        json_script = pre_tag.text

        # Parse the JSON data
        json_parsed = json.loads(json_script)

        # Add json to the list
        json_list.append(json_parsed)
    except (NoSuchElementException, TimeoutException, json.JSONDecodeError) as e:
        print(f"Error processing URL {url}: {e}")


In [15]:
# Lists to store URLs and corresponding dates
all_urls = []
all_dates = []

# Iterate through each JSON data in the list
for json_data in json_list:
    
    # Extract the list from the 'items' key
    item_list = json_data.get('items', [])
    
    # Iterate through each item in the 'items' list
    for item in item_list:
        
        # Extract the date the item was taken
        date_taken = item.get('taken_at')  # Move this line inside the loop

        # Check if 'carousel_media' is present
        carousel_media = item.get('carousel_media', [])
        
        # Iterate through each media in the 'carousel_media' list
        for media in carousel_media:
            
            # Extract the image URL from the media
            image_url = media.get('image_versions2', {}).get('candidates', [{}])[0].get('url')
            
            if image_url:
                # Add the image URL and corresponding date to the lists
                all_urls.append(image_url)
                all_dates.append(date_taken)
                print(f"carousel image added")
                
            # Extract the video URL from the media
            video_versions = media.get('video_versions', [])
            if video_versions:
                video_url = video_versions[0].get('url')
                if video_url:
                    
                    # Add the video URL and corresponding date to the lists
                    all_urls.append(video_url)
                    all_dates.append(date_taken)
                    print(f"carousel video added")

        # Handle cases of unique image, instead of carousel
        image_url = item.get('image_versions2', {}).get('candidates', [{}])[0].get('url')
        if image_url:
            
            # Add the image URL and corresponding date to the lists
            all_urls.append(image_url)
            all_dates.append(date_taken)
            print(f"single image added")

        # Check if 'video_versions' key exists
        video_versions = item.get('video_versions', [])
        if video_versions:
            video_url = video_versions[0].get('url')
            if video_url:
                all_urls.append(video_url)
                all_dates.append(date_taken)
                print(f"video added")
                
# Print or use all collected URLs as needed
print(len(all_urls))
                



NameError: name 'json_list' is not defined

In [28]:
import requests
import os

# Fonction pour télécharger un fichier
def download_file(url, file_path):
    response = requests.get(url, stream=True)
    with open(file_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                file.write(chunk)
    print(f"Download : {file_path}")


def download_instagram_post(post_url):
    shortcode = post_url.split("/")[-2]

    api_url = f"https://www.instagram.com/p/{shortcode}/?__a=1&__d=dis"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
    }
    response = requests.get(api_url, headers=headers)
    
    if response.status_code != 200:
        print("Impossible to fetch post data.")
        return

    media_data = response.json()['graphql']['shortcode_media']
    
    download_dir = "Downloaded"
    os.makedirs(download_dir, exist_ok=True)

    if media_data['is_video']:
        video_url = media_data['video_url']
        file_path = os.path.join(download_dir, f"{shortcode}.mp4")
        download_file(video_url, file_path)
    else:
        if 'edge_sidecar_to_children' in media_data:
            for edge in media_data['edge_sidecar_to_children']['edges']:
                node = edge['node']
                if node['is_video']:
                    video_url = node['video_url']
                    file_path = os.path.join(download_dir, f"{node['id']}.mp4")
                    download_file(video_url, file_path)
                else:
                    image_url = node['display_url']
                    file_path = os.path.join(download_dir, f"{node['id']}.jpg")
                    download_file(image_url, file_path)
        else:
            image_url = media_data['display_url']
            file_path = os.path.join(download_dir, f"{shortcode}.jpg")
            download_file(image_url, file_path)

post_url = "https://www.instagram.com/reel/Ct1ETw7AglH/?utm_source=ig_web_copy_link&igsh=MzRlODBiNWFlZA=="
download_instagram_post(post_url)


Téléchargé : DWN/Ct1ETw7AglH.mp4


In [30]:
def convert_video_to_audio(shortcode):
    video = VideoFileClip(f"{shortcode}.mp4")
    video.audio.write_audiofile("audio.mp3")

OSError: MoviePy error: the file example.mp4 could not be found!
Please check that you entered the correct path.